In [1]:
import requests
import configparser
import telegram
from flask import Flask, request
from telegram.ext import Dispatcher, MessageHandler, Filters

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [3]:
config['TELEGRAM']['ACCESS_TOKEN']

'845046416:AAHy4LTI3e5sZqeOEvIhhtMKIwkCaL5zkvc'

In [4]:
config['TELEGRAM']['WEBHOOK_URL']

'https://af179f9b.ngrok.io'

In [5]:
access_token = config['TELEGRAM']['ACCESS_TOKEN']
webhook_url = config['TELEGRAM']['WEBHOOK_URL']

## delete webhook url

In [47]:
requests.post('https://api.telegram.org/bot'+access_token+'/deleteWebhook').text


'{"ok":true,"result":true,"description":"Webhook was deleted"}'

## set webhook url

In [48]:
requests.post('https://api.telegram.org/bot'+access_token+'/setWebhook?url='+webhook_url+'/hook').text

'{"ok":true,"result":true,"description":"Webhook was set"}'

# 三個function

In [49]:
# Initial Flask app
# coding: utf-8
from fugle_realtime_restful_api import *
import datetime

app = Flask(__name__)

# Initial bot by Telegram access token
bot = telegram.Bot(token=config['TELEGRAM']['ACCESS_TOKEN'])
def meta(sym):
    api='b71c49b4c7925bb66e6f59f068f0892f'
    c=intraday.meta(apiToken=api, output='raw', symbolId=sym)
    
    nameZhTw = c['nameZhTw']#股票中文簡稱
    industryZhTw = c['industryZhTw'] #產業別

    typeZhTw = c['typeZhTw']#股票類別
    isIndex = c['isIndex'] #是否為指數
    isWarrant = c['isWarrant']#是否為權證
    isSuspended = c['isSuspended'] #今日是否暫停買賣
    isTerminated = c['isTerminated']#今日是否已終止上市

    priceHighLimit = c['priceHighLimit']#漲停價
    priceLowLimit = c['priceLowLimit']#跌停價
    priceReference = c['priceReference']#今日參考價

    canDayBuySell = c['canDayBuySell'] #是否可先買後賣現股當沖
    canDaySellBuy = c['canDaySellBuy'] #是否可先賣後買現股當沖
    canShortLend = c['canShortLend']   #是否豁免平盤下融券賣出
    canShortMargin = c['canShortMargin']#是否豁免平盤下借券賣出
    
    x=[nameZhTw,industryZhTw,
       typeZhTw,str(isIndex),str(isWarrant),str(isSuspended),str(isTerminated),
       priceHighLimit,priceLowLimit,priceReference,
       str(canDayBuySell),str(canDaySellBuy),str(canShortLend),str(canShortMargin)]
    return x


def chart(sym):
    def time():
        x = datetime.datetime.now()
        a1 = x.year 
        a2 = x.month 
        a3 = x.day 
        a4 = x.hour -8
        a5 = x.minute
        if a2 < 9 :
            a2 = '0'+str(a2)
        if a3 < 9 :
            a3 = '0'+str(a3)
        if a4 < 0 :
            a4 = '00'
        elif a4 < 9 :
            a4 = '0'+str(a4)
        if a5 < 9 :
            a5 = '0'+str(a5)
        r =str(a1)+'-'+str(a2)+'-'+str(a3)+'T'+str(a4)+':'+str(a5)+':00.000Z'
        return r
    api='b71c49b4c7925bb66e6f59f068f0892f'
    c=intraday.chart(apiToken=api, output='raw', symbolId=sym)
    x=time()
    try:
        a = c[x]
        xclose = a['close']
        xhigh = a['high']
        xlow = a['low']
        xopen = a['open']
        xunit = a['unit']
        xvolume = a['volume']
        xx=[xopen,xhigh,xlow,xclose,xunit,xvolumn]
        return xx
    except:
        return '非股票開盤時間'
    
    
def quote(sym):
    api='b71c49b4c7925bb66e6f59f068f0892f'
    c=intraday.quote(apiToken=api, output='raw', symbolId = sym)

    isOpenDelayed = c['isOpenDelayed'] #當日是否曾發生延後開盤
    isCloseDelayed = c['isCloseDelayed'] #當日是否曾發生延後收盤
    isClosed = c['isClosed'] #當日是否為已收盤
    isHalting = c['isHalting'] #最近一次更新是否為暫停交易
    isCurbing = c['isCurbing'] #最近一次更新是否為熔斷
    isTrial = c['isTrial'] #最近一次更新是否為試算


    total = c['total'] 
    #t_order = c['total']['order'] #總成交委託,負數表示無 order
    #t_price = c['total']['price'] #總成交價,負數表示無 price
    t_unit = c['total']['unit'] #總成交張數
    t_volume = c['total']['volume'] #總成交量
    priceHigh = c['priceHigh']['price'] #當日之最高價
    priceLow = c['priceLow']['price'] #當日之最低價
    priceOpen = c['priceOpen']['price'] #當日之開盤價

    trial = c['trial'] #最新一筆試撮
    trade = c['trade'] #最新一筆成交
    bestAsks = order['bestAsks'] #最新一筆最佳五檔
    bestBids = order['bestBids']
    
    x = [isOpenDelayed , isCloseDelayed ,isClosed , isHalting , isCurbing,
        isTrial]
    #y = [t_order , t_price , t_unit , t_volume , priceHigh , priceLow ,
    #     priceOpen]
    y = [t_unit , t_volume , priceHigh , priceLow ,priceOpen]
    z = [trial , trade , bestAsks , bestBids]
    
    return x,y,z



@app.route('/hook', methods=['POST'])
def webhook_handler():#接收訊息
    if request.method == "POST":
        update = telegram.Update.de_json(request.get_json(force=True), bot)

        # Update dispatcher process that handler to process this message
        dispatcher.process_update(update)
    return 'ok'

## reply message
def reply_handler(bot, update):#回覆訊息
    text = update.message.text
    user_id = update.message.from_user.name
    
    if text[0] == '1':
        text = text[2:6]
        x = meta(text)
        a,b,c = quote(text)
        update.message.reply_text('股票中文簡稱：'+ x[0]+
                                      '\n產業別：'+ x[1]+
                                      '\n\n股票類別：'+ x[2] +
                                      '\n是否為指數：'+str(x[3])+
                                      '\n是否為權證：'+ str(x[4])+
                                      '\n今日是否暫停買賣：'+ str(x[5])+
                                      '\n今日是否已終止上市：'+ str(x[6]) +
                                      '\n\n漲停價：'+ str(x[7]) +
                                      '\n跌停價：'+ str(x[8]) +
                                      '\n今日參考價：'+ str(x[9]) +
                                      '\n\n是否可先買後賣現股當沖：'+ str(x[10]) +
                                      '\n是否可先賣後買現股當沖：'+str( x[11]) +
                                      '\n是否豁免平盤下融券賣出：'+ str(x[12]) +
                                      '\n是否豁免平盤下借券賣出：'+ str(x[13]) +
                                      '\n\n當日是否曾發生延後開盤：'+ str(a[0]) +
                                      '\n當日是否曾發生延後收盤：'+ str(a[1]) +
                                      '\n當日是否為已收盤：'+ str(a[2]) +
                                      '\n最近一次更新是否為暫停交易：'+ str(a[3]) +
                                      '\n最近一次更新是否為熔斷：'+ str(a[4]) +
                                      '\n最近一次更新是否為試算：'+ str(a[5])
                                     )
        
    elif text[0] == '2':
        text = text[2:6]
        x = chart(text)
        a,b,c = quote(text)
        if x == '非股票開盤時間':
            update.message.reply_text('非股票開盤時間')
        else:
            update.message.reply_text('此分鐘的開盤價'+ str(x[0])+
                                        '\n此分鐘的最高價：'+ str(x[1])+
                                        '\n此分鐘的最低價：'+ str(x[2])+
                                        '\n此分鐘的收盤價：'+ str(x[3])+
                                        '\n\n此分鐘的交易張數：'+str(x[4])+
                                        '\n此分鐘的交易量：'+str(x[5])+
                                        '\n\n最新一筆試撮：'+str(c[0])+
                                        '\n最新一筆成交：'+str(c[1])+
                                        '\n最新一筆最佳五檔 bestAsks：'+str(c[2])+
                                        '\n最新一筆最佳五檔 bestBids：'+str(c[3])
                                     )
            
    elif text[0] == '3':
        text = text[2:6]
        a,b,c = quote(text)
        update.message.reply_text('\n總成交張數：'+ str(b[0])+
                                  '\n總成交量：'+ str(b[1])+
                                  '\n\n當日之最高價：'+str(b[2])+
                                  '\n當日之最低價：'+str(b[3])+
                                  '\n當日之開盤價：'+str(b[4])
                                 )
        
    elif text[0] == '4':
        update.message.reply_text('hi hi hi!!!')
    
    else:
        update.message.reply_text('需要什麼服務？\n'+
                                 '範例：\n'+
                                 '輸入 {1 股票代碼}: 查詢個股資訊\n'+
                                 '輸入 {2 股票代碼}: 查詢個股即時交易資料\n'+
                                 '輸入 {3 股票代碼}: 查詢個股當日交易資料\n'
                                 )
        
        
        

    
    
# This class dispatches all kinds of updates to its registered handlers.
dispatcher = Dispatcher(bot, None)#管理所有設立的handler
dispatcher.add_handler(MessageHandler(Filters.text, reply_handler))

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2020 22:43:18] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 22:43:23] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 22:43:30] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 22:43:41] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 22:43:47] "POST /hook HTTP/1.1" 200 -


In [8]:
# Initial Flask app
# coding: utf-8
from fugle_realtime_restful_api import *
import datetime

app = Flask(__name__)

# Initial bot by Telegram access token
bot = telegram.Bot(token=config['TELEGRAM']['ACCESS_TOKEN'])
def meta(sym):
    api='b71c49b4c7925bb66e6f59f068f0892f'
    c=intraday.meta(apiToken=api, output='raw', symbolId=sym)
    
    nameZhTw = c['nameZhTw']#股票中文簡稱
    industryZhTw = c['industryZhTw'] #產業別

    typeZhTw = c['typeZhTw']#股票類別
    isIndex = c['isIndex'] #是否為指數
    isWarrant = c['isWarrant']#是否為權證
    isSuspended = c['isSuspended'] #今日是否暫停買賣
    isTerminated = c['isTerminated']#今日是否已終止上市

    priceHighLimit = c['priceHighLimit']#漲停價
    priceLowLimit = c['priceLowLimit']#跌停價
    priceReference = c['priceReference']#今日參考價

    canDayBuySell = c['canDayBuySell'] #是否可先買後賣現股當沖
    canDaySellBuy = c['canDaySellBuy'] #是否可先賣後買現股當沖
    canShortLend = c['canShortLend']   #是否豁免平盤下融券賣出
    canShortMargin = c['canShortMargin']#是否豁免平盤下借券賣出
    
    x=[nameZhTw,industryZhTw,
       typeZhTw,str(isIndex),str(isWarrant),str(isSuspended),str(isTerminated),
       priceHighLimit,priceLowLimit,priceReference,
       str(canDayBuySell),str(canDaySellBuy),str(canShortLend),str(canShortMargin)]
    return x
def chart(sym):
    def time():
        x = datetime.datetime.now()
        a1 = x.year 
        a2 = x.month 
        a3 = x.day 
        a4 = x.hour+8
        a5 = x.minute
        if a2 < 9 :
            a2 = '0'+str(a2)
        if a3 < 9 :
            a3 = '0'+str(a3)
        if a4 < 9 :
            a4 = '0'+str(a4)
        elif a4 > 24 : 
            a4 = a4 - 24
        if a5 < 9 :
            a5 = '0'+str(a5)
        r =str(a1)+'-'+str(a2)+'-'+str(a3)+'T'+str(a4)+':'+str(a5)+':00.000Z'
        return r
    api='b71c49b4c7925bb66e6f59f068f0892f'
    c=intraday.chart(apiToken=api, output='raw', symbolId=sym)
    x=time()
    try:
        a = c[x]
        xclose = a['close']
        xhigh = a['high']
        xlow = a['low']
        xopen = a['open']
        xunit = a['unit']
        xvolume = a['volume']
        xx=[xopen,xhigh,xlow,xclose,xunit,xvolumn]
        return xx
    except:
        return '非股票開盤時間'



@app.route('/hook', methods=['POST'])
def webhook_handler():#接收訊息
    if request.method == "POST":
        update = telegram.Update.de_json(request.get_json(force=True), bot)

        # Update dispatcher process that handler to process this message
        dispatcher.process_update(update)
    return 'ok'

## reply message
def reply_handler(bot, update):#回覆訊息
    text = update.message.text
    user_id = update.message.from_user.name
    
    if text[0] == '1':
        text = text[2:6]
        x = meta(text)
        update.message.reply_text('股票中文簡稱：'+ x[0]+
                                      '\n產業別：'+ x[1]+
                                      '\n\n股票類別：'+ x[2] +
                                      '\n是否為指數：'+str(x[3])+
                                      '\n是否為權證：'+ str(x[4])+
                                      '\n今日是否暫停買賣：'+ str(x[5])+
                                      '\n今日是否已終止上市：'+ str(x[6]) +
                                      '\n\n漲停價：'+ str(x[7]) +
                                      '\n跌停價：'+ str(x[8]) +
                                      '\n今日參考價：'+ str(x[9]) +
                                      '\n\n是否可先買後賣現股當沖：'+ str(x[10]) +
                                      '\n是否可先賣後買現股當沖：'+str( x[11]) +
                                      '\n是否豁免平盤下融券賣出：'+ str(x[12]) +
                                      '\n是否豁免平盤下借券賣出：'+ str(x[13])
                                     )
    elif text[0] == '2':
        text = text[2:6]
        x = chart(text)
        if x == '非股票開盤時間':
            update.message.reply_text('非股票開盤時間')
        else:
            update.message.reply_text('此分鐘的開盤價'+ str(x[0])+
                                      '\n此分鐘的最高價'+ str(x[1])+
                                      '\n此分鐘的最低價'+ str(x[2])+
                                      '\n此分鐘的收盤價'+ str(x[3])+
                                      '\n\n此分鐘的交易張數'+str(x[4])+
                                      '\n此分鐘的交易量'+str(x[5])
                                     )
    elif text[0] == '3':
        update.message.reply_text('hi')
    
    else:
        update.message.reply_text('需要什麼服務？\n'+
                                 '範例：\n'+
                                 '輸入 {1 股票代碼}: 查詢個股資訊\n'+
                                 '輸入 {2 股票代碼}: 查詢個股即時交易資料\n'+
                                 '輸入 {3 股票代碼}: 查詢個股最佳Bit price\n')
        
        
        

    
    
# This class dispatches all kinds of updates to its registered handlers.
dispatcher = Dispatcher(bot, None)#管理所有設立的handler
dispatcher.add_handler(MessageHandler(Filters.text, reply_handler))

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/opt/anaconda3/envs/fugle/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: TelegramDeprecationWarning:

Old Handler API is deprecated - see https://git.io/fxJuV for details

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2020 20:46:13] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 20:46:27] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 20:46:34] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 20:46:37] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 20:46:40] "POST /hook HTTP/1.1" 200 -


In [79]:
# Initial Flask app
# coding: utf-8
from fugle_realtime_restful_api import *
import datetime

app = Flask(__name__)

# Initial bot by Telegram access token
bot = telegram.Bot(token=config['TELEGRAM']['ACCESS_TOKEN'])
def meta(sym):
    api='b71c49b4c7925bb66e6f59f068f0892f'
    c=intraday.meta(apiToken=api, output='raw', symbolId=sym)
    
    nameZhTw = c['nameZhTw']#股票中文簡稱
    industryZhTw = c['industryZhTw'] #產業別

    typeZhTw = c['typeZhTw']#股票類別
    isIndex = c['isIndex'] #是否為指數
    isWarrant = c['isWarrant']#是否為權證
    isSuspended = c['isSuspended'] #今日是否暫停買賣
    isTerminated = c['isTerminated']#今日是否已終止上市

    priceHighLimit = c['priceHighLimit']#漲停價
    priceLowLimit = c['priceLowLimit']#跌停價
    priceReference = c['priceReference']#今日參考價

    canDayBuySell = c['canDayBuySell'] #是否可先買後賣現股當沖
    canDaySellBuy = c['canDaySellBuy'] #是否可先賣後買現股當沖
    canShortLend = c['canShortLend']   #是否豁免平盤下融券賣出
    canShortMargin = c['canShortMargin']#是否豁免平盤下借券賣出
    
    x=[nameZhTw,industryZhTw,
       typeZhTw,str(isIndex),str(isWarrant),str(isSuspended),str(isTerminated),
       priceHighLimit,priceLowLimit,priceReference,
       str(canDayBuySell),str(canDaySellBuy),str(canShortLend),str(canShortMargin)]
    return x
def chart(sym):
    def time():
        x = datetime.datetime.now()
        a1 = x.year 
        a2 = x.month 
        a3 = x.day 
        a4 = x.hour+8
        a5 = x.minute
        if a2 < 9 :
            a2 = '0'+str(a2)
        if a3 < 9 :
            a3 = '0'+str(a3)
        if a4 < 9 :
            a4 = '0'+str(a4)
        elif a4 > 24 : 
            a4 = a4 - 24
        if a5 < 9 :
            a5 = '0'+str(a5)
        r =str(a1)+'-'+str(a2)+'-'+str(a3)+'T'+str(a4)+':'+str(a5)+':00.000Z'
        return r
    api='b71c49b4c7925bb66e6f59f068f0892f'
    c=intraday.chart(apiToken=api, output='raw', symbolId=sym)
    x=time()
    try:
        a = c[x]
        xclose = a['close']
        xhigh = a['high']
        xlow = a['low']
        xopen = a['open']
        xunit = a['unit']
        xvolume = a['volume']
        xx=[xopen,xhigh,xlow,xclose,xunit,xvolumn]
        return xx
    except:
        return '非股票開盤時間'



@app.route('/hook', methods=['POST'])
def webhook_handler():#接收訊息
    if request.method == "POST":
        update = telegram.Update.de_json(request.get_json(force=True), bot)

        # Update dispatcher process that handler to process this message
        dispatcher.process_update(update)
    return 'ok'

## reply message
def reply_handler(bot, update):#回覆訊息
    text = update.message.text
    user_id = update.message.from_user.name
    text = text[2:6]
    if text[0] == '1':
        x = meta(text)
        update.message.reply_text('股票中文簡稱：'+ x[0]+
                                      '\n產業別：'+ x[1]+
                                      '\n\n股票類別：'+ x[2] +
                                      '\n是否為指數：'+str(x[3])+
                                      '\n是否為權證：'+ str(x[4])+
                                      '\n今日是否暫停買賣：'+ str(x[5])+
                                      '\n今日是否已終止上市：'+ str(x[6]) +
                                      '\n\n漲停價：'+ str(x[7]) +
                                      '\n跌停價：'+ str(x[8]) +
                                      '\n今日參考價：'+ str(x[9]) +
                                      '\n\n是否可先買後賣現股當沖：'+ str(x[10]) +
                                      '\n是否可先賣後買現股當沖：'+str( x[11]) +
                                      '\n是否豁免平盤下融券賣出：'+ str(x[12]) +
                                      '\n是否豁免平盤下借券賣出：'+ str(x[13])
                                     )
    elif text[0] == '2':
        x = chart(text)
        if x == '非股票開盤時間':
            update.message.reply_text('非股票開盤時間')
        else:
            update.message.reply_text('此分鐘的開盤價'+ str(x[0])+
                                      '\n此分鐘的最高價'+ str(x[1])+
                                      '\n此分鐘的最低價'+ str(x[2])+
                                      '\n此分鐘的收盤價'+ str(x[3])+
                                      '\n\n此分鐘的交易張數'+str(x[4])+
                                      '\n此分鐘的交易量'+str(x[5])
                                     )
    elif text[0] == '3':
        update.message.reply_text('hi')
    
    else:
        update.message.reply_text('需要什麼服務？\n'+
                                 '範例：\n'+
                                 '輸入 {1 股票代碼}: 查詢個股資訊\n'+
                                 '輸入 {2 股票代碼}: 查詢個股即時交易資料\n'+
                                 '輸入 {3 股票代碼}: 查詢個股最佳Bit price\n'
                                 )
        
        
        

    
    
# This class dispatches all kinds of updates to its registered handlers.
dispatcher = Dispatcher(bot, None)#管理所有設立的handler
dispatcher.add_handler(MessageHandler(Filters.text, reply_handler))

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/opt/anaconda3/envs/fugle/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: TelegramDeprecationWarning:

Old Handler API is deprecated - see https://git.io/fxJuV for details

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Apr/2020 02:44:22] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 02:44:41] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 02:45:11] "POST /hook HTTP/1.1" 200 -


In [31]:
w = c['trial']
str(w)
w

{'at': '2020-04-01T05:29:59.086Z',
 'price': 162.5,
 'unit': 493,
 'volume': 493000}

In [80]:
r

NameError: name 'r' is not defined

In [ ]:
a2=a.loc[0]
a2

In [39]:
 def asdf(sym):
    api='b71c49b4c7925bb66e6f59f068f0892f'
    c=intraday.meta(apiToken=api, output='raw', symbolId=sym)
    
    nameZhTw = c['nameZhTw']#股票中文簡稱
    industryZhTw = c['industryZhTw'] #產業別
    print(industryZhTw)
    print(type(industryZhTw))

    typeZhTw = c['typeZhTw']#股票類別
    isIndex = c['isIndex'] #是否為指數
    isWarrant = c['isWarrant']#是否為權證
    isSuspended = c['isSuspended'] #今日是否暫停買賣
    isTerminated = c['isTerminated']#今日是否已終止上市
    print(type(str(isIndex)))
    print(type(isIndex))
    

    priceHighLimit = c['priceHighLimit']#漲停價
    priceLowLimit = c['priceLowLimit']#跌停價
    priceReference = c['priceReference']#今日參考價

    canDayBuySell = c['canDayBuySell'] #是否可先買後賣現股當沖
    canDaySellBuy = c['canDaySellBuy'] #是否可先賣後買現股當沖
    canShortLend = c['canShortLend']   #是否豁免平盤下融券賣出
    canShortMargin = c['canShortMargin']#是否豁免平盤下借券賣出
asdf('2492')

電子零組件業
<class 'str'>
<class 'str'>
<class 'bool'>


In [63]:
api='b71c49b4c7925bb66e6f59f068f0892f'
c=intraday.chart(apiToken=api, output='raw', symbolId='2492')
c

{'2020-04-01T01:01:00.000Z': {'close': 159.5,
  'high': 161,
  'low': 159.5,
  'open': 160.5,
  'unit': 282,
  'volume': 282000},
 '2020-04-01T01:02:00.000Z': {'close': 158.5,
  'high': 160,
  'low': 158.5,
  'open': 159.5,
  'unit': 106,
  'volume': 106000},
 '2020-04-01T01:03:00.000Z': {'close': 158.5,
  'high': 159,
  'low': 158.5,
  'open': 158.5,
  'unit': 171,
  'volume': 171000},
 '2020-04-01T01:04:00.000Z': {'close': 158.5,
  'high': 159,
  'low': 158,
  'open': 158.5,
  'unit': 146,
  'volume': 146000},
 '2020-04-01T01:05:00.000Z': {'close': 158.5,
  'high': 159,
  'low': 158,
  'open': 158.5,
  'unit': 46,
  'volume': 46000},
 '2020-04-01T01:06:00.000Z': {'close': 159,
  'high': 159.5,
  'low': 158.5,
  'open': 158.5,
  'unit': 78,
  'volume': 78000},
 '2020-04-01T01:07:00.000Z': {'close': 159.5,
  'high': 160,
  'low': 159,
  'open': 159,
  'unit': 59,
  'volume': 59000},
 '2020-04-01T01:08:00.000Z': {'close': 160,
  'high': 160,
  'low': 159.5,
  'open': 160,
  'unit': 28,


In [54]:
import datetime
x = datetime.datetime.now() #現在時間
#x = datetime.datetime(2008, 12, 5, 23, 59, 59) #指定時間
x.year #會拿到 2008
x.month #會拿到 12
x.day # 會拿到 5
x.hour   #時
x.minute #分
x.second #秒 59

datetime.datetime(2020, 4, 4, 1, 49, 33, 304295)

In [78]:
import datetime
def time():
    x = datetime.datetime.now()
    a1 = x.year 
    a2 = x.month 
    a3 = x.day 
    a4 = x.hour+8
    a5 = x.minute
    if a2 < 9 :
        a2 = '0'+str(a2)
    if a3 < 9 :
        a3 = '0'+str(a3)
    if a4 < 9 :
        a4 = '0'+str(a4)
    elif a4 > 24 : 
        a4 = a4 - 24
    if a5 < 9 :
        a5 = '0'+str(a5)
    r =str(a1)+'-'+str(a2)+'-'+str(a3)+'T'+str(a4)+':'+str(a5)+':00.000Z'
    return r
x=time()
x

'2020-04-04T10:36:00.000Z'

In [60]:
import datetime
def time():
    x = datetime.datetime.now()
    a1 = x.year 
    a2 = x.month 
    a3 = x.day 
    a4 = x.hour
    a5 = x.minute
    if a2 < 9 :
        a2 = '0'+str(a2)
    if a3 < 9 :
        a3 = '0'+str(a3)
    if a4 < 9 :
        a4 = '0'+str(a4)
    if a5 < 9 :
        a5 = '0'+str(a5)
    r =str(a1)+'-'+str(a2)+'-'+str(a3)+'T'+str(a4)+':'+str(a5)+':00.000Z'
    return r


api='b71c49b4c7925bb66e6f59f068f0892f'
c=intraday.chart(apiToken=api, output='raw', symbolId='2492')
x=time()
try:
    a = c[x]
    xclose = a['close']
    xhigh = a['high']
    xlow = a['low']
    xopen = a['open']
    xunit = a['unit']
    xvolume = a['volume']
    xx=[xclose,xhigh,xlow,xopen,xunit,xvolumn]
    return xx
except:
    print('非股票開盤時間')
    


'2020-04-04T01:56:00.000Z'

In [70]:
c[x]

{'close': 159.5,
 'high': 161,
 'low': 159.5,
 'open': 160.5,
 'unit': 282,
 'volume': 282000}

In [68]:
from datetime import datetime
my_date = datetime.now()
print(my_date.isoformat())

2020-04-04T02:07:23.247180


In [72]:
a='1234'
a[0]

'1'

In [75]:
import datetime
def chart(sym):
    def time():
        x = datetime.datetime.now()
        a1 = x.year 
        a2 = x.month 
        a3 = x.day 
        a4 = x.hour
        a5 = x.minute
        if a2 < 9 :
            a2 = '0'+str(a2)
        if a3 < 9 :
            a3 = '0'+str(a3)
        if a4 < 9 :
            a4 = '0'+str(a4)
        if a5 < 9 :
            a5 = '0'+str(a5)
        r =str(a1)+'-'+str(a2)+'-'+str(a3)+'T'+str(a4)+':'+str(a5)+':00.000Z'
        return r
    api='b71c49b4c7925bb66e6f59f068f0892f'
    c=intraday.chart(apiToken=api, output='raw', symbolId=sym)
    x=time()
    try:
        a = c[x]
        xclose = a['close']
        xhigh = a['high']
        xlow = a['low']
        xopen = a['open']
        xunit = a['unit']
        xvolume = a['volume']
        xx=[xclose,xhigh,xlow,xopen,xunit,xvolumn]
        return xx
    except:
        return' 非股票開盤時間'

x = chart('2492')
x

' 非股票開盤時間'

In [15]:
api='b71c49b4c7925bb66e6f59f068f0892f'
c=intraday.quote(apiToken=api, output='raw', symbolId='2492')
c

{'isCloseDelayed': False,
 'isClosed': True,
 'isHalting': False,
 'isOpenDelayed': False,
 'order': {'bestAsks': [{'price': 162.5, 'unit': 418, 'volume': 418000},
   {'price': 163, 'unit': 268, 'volume': 268000},
   {'price': 163.5, 'unit': 97, 'volume': 97000},
   {'price': 164, 'unit': 156, 'volume': 156000},
   {'price': 164.5, 'unit': 108, 'volume': 108000}],
  'bestBids': [{'price': 160, 'unit': 204, 'volume': 204000},
   {'price': 160.5, 'unit': 159, 'volume': 159000},
   {'price': 161, 'unit': 129, 'volume': 129000},
   {'price': 161.5, 'unit': 36, 'volume': 36000},
   {'price': 162, 'unit': 10, 'volume': 10000}],
  'at': '2020-04-01T05:30:00.000Z'},
 'priceOpen': {'at': '2020-04-01T01:00:02.233Z', 'price': 160.5},
 'isCurbing': False,
 'isTrial': False,
 'total': {'at': '2020-04-01T05:55:11.502Z', 'unit': 8916, 'volume': 8916000},
 'trial': {'at': '2020-04-01T05:29:59.086Z',
  'price': 162.5,
  'unit': 493,
  'volume': 493000},
 'priceHigh': {'at': '2020-04-01T01:54:37.624Z', 

In [18]:
api='b71c49b4c7925bb66e6f59f068f0892f'
c=intraday.quote(apiToken=api, output='raw', symbolId='2492')

isOpenDelayed = c['isOpenDelayed']
isCloseDelayed = c['isCloseDelayed']
isClosed = c['isClosed']
isHalting = c['isHalting']
isCurbing = c['isCurbing'] #最近一次更新是否為熔斷
isTrial = c['isTrial'] #最近一次更新是否為試算


total = c['total'] 
t_order = c['total']['order'] #總成交委託, 負數表示無 order
t_price = c['total']['price'] #總成交價, 負數表示無 price
t_unit = c['total']['unit'] #總成交張數
t_volume = c['total']['volume'] #總成交量

trial = c['trial'] #最新一筆試撮
trade = c['trade'] #最新一筆成交

order = c['order'] #總成交委託, 負數表示無 order
bestAsks = order['bestAsks'] #最新一筆最佳五檔
bestBids = order['bestBids']


priceHigh = c['priceHigh'] #當日之最高價
priceLow = c['priceLow']['price'] #當日之最低價
priceOpen = c['priceOpen']['price'] #當日之開盤價   

In [40]:
total

{'at': '2020-04-01T05:55:11.502Z', 'unit': 8916, 'volume': 8916000}

In [18]:
def quote(sym)
    api='b71c49b4c7925bb66e6f59f068f0892f'
    c=intraday.quote(apiToken=api, output='raw', symbolId = sym)

    isOpenDelayed = c['isOpenDelayed']
    isCloseDelayed = c['isCloseDelayed']
    isClosed = c['isClosed']
    isHalting = c['isHalting']
    isCurbing = c['isCurbing'] #最近一次更新是否為熔斷
    isTrial = c['isTrial'] #最近一次更新是否為試算


    total = c['total'] 
    t_order = c['total']['order'] #總成交委託,負數表示無 order
    t_price = c['total']['price'] #總成交價,負數表示無 price
    t_unit = c['total']['unit'] #總成交張數
    t_volume = c['total']['volume'] #總成交量
    priceHigh = c['priceHigh']['price'] #當日之最高價
    priceLow = c['priceLow']['price'] #當日之最低價
    priceOpen = c['priceOpen']['price'] #當日之開盤價

    trial = c['trial'] #最新一筆試撮
    trade = c['trade'] #最新一筆成交
    bestAsks = order['bestAsks'] #最新一筆最佳五檔
    bestBids = order['bestBids']
    
    x = [isOpenDelayed , isCloseDelayed ,isClosed , isHalting , isCurbing,
        isTrial]
    y = [total , t_order , t_price , t_unit , t_volume , priceHigh , priceLow , priceOpen]
    z = [trial , trade , bestAsks , bestBids]
    
    return x,y,z

In [19]:
order

{'bestAsks': [{'price': 162.5, 'unit': 418, 'volume': 418000},
  {'price': 163, 'unit': 268, 'volume': 268000},
  {'price': 163.5, 'unit': 97, 'volume': 97000},
  {'price': 164, 'unit': 156, 'volume': 156000},
  {'price': 164.5, 'unit': 108, 'volume': 108000}],
 'bestBids': [{'price': 160, 'unit': 204, 'volume': 204000},
  {'price': 160.5, 'unit': 159, 'volume': 159000},
  {'price': 161, 'unit': 129, 'volume': 129000},
  {'price': 161.5, 'unit': 36, 'volume': 36000},
  {'price': 162, 'unit': 10, 'volume': 10000}],
 'at': '2020-04-01T05:30:00.000Z'}

In [29]:
o = str(order['bestAsks'])
o

"[{'price': 162.5, 'unit': 418, 'volume': 418000}, {'price': 163, 'unit': 268, 'volume': 268000}, {'price': 163.5, 'unit': 97, 'volume': 97000}, {'price': 164, 'unit': 156, 'volume': 156000}, {'price': 164.5, 'unit': 108, 'volume': 108000}]"

In [22]:
order['bestAsks'][0]


{'price': 162.5, 'unit': 418, 'volume': 418000}

In [23]:
order['bestAsks'][0]['price']

162.5

In [ ]:
ap1 = order['bestAsks'][0]['price']
ap2 = order['bestAsks'][1]['price']
ap3 = order['bestAsks'][2]['price']
ap4 = order['bestAsks'][3]['price']
ap5 = order['bestAsks'][4]['price']

au1 = order['bestAsks'][0]['unit']
au2 = order['bestAsks'][1]['unit']
au3 = order['bestAsks'][2]['unit']
au4 = order['bestAsks'][3]['unit']
au5 = order['bestAsks'][4]['unit']

av1 = order['bestAsks'][0]['volumn']
av2 = order['bestAsks'][1]['volumn']
av3 = order['bestAsks'][2]['volumn']
av4 = order['bestAsks'][3]['volumn']
av5 = order['bestAsks'][4]['volumn']